# Single channel vs classic gateways

In [ ]:
# SET your current working directory!
cwd = "/home/jovyan/work/energy"
overwrite = False

In [ ]:
import os
import sem
import numpy as np
import scipy as sp
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.style.use(["science", "ieee"])
plt.rcParams.update({"figure.dpi": "200"})

# Create our SEM campaign
ns_3_dir = os.environ["NS3DIR"]
script = "clues"
results_dir = cwd + "/results"
plots_dir = cwd + "/plots"

os.makedirs(plots_dir, exist_ok=True)
campaign = sem.CampaignManager.new(ns_3_dir, script, results_dir, overwrite=overwrite)
overwrite = False
runs = 30

In [ ]:
# If the previous cell fails due to changes in the simulation code,
# you can re-run the previous cell after running this one to fix it.
##################### WARNING! #####################
# It will overwrite all existing simulation results,
# so back them up beforehand if you need them.
overwrite = True

Utility functions

In [ ]:
def plot_size(h, gold=False):
    letter = 1.294
    r = sp.constants.golden if gold else letter
    w = h * r
    return (w, h)

## Run experiments

In [ ]:
# Define the parameter space we are interested in exploring
params = {
    "hoursNum": 10,
    "sideLength": 9200,
    "devNum": 10000,
    "gwNum": list(range(1, 11, 1)),
    "scenario": ["SF7Single", "FullRand", "Classic"],
}

# Run simulations with the above parameter space
print(f"• Running missing simulations for param space: {params}.")
campaign.run_missing_simulations(params, runs)

## LoRa transmitters' energy consumption

In [ ]:
@sem.utils.output_labels(["TotalEnergyJ"])
def get_energy(result):
    """
    Extract the total energy consumption from energy.csv
    """
    return pd.read_csv(StringIO(result["output"]["energy.csv"]))["TotalEnergyJ"].iloc[0]


filename = "energy"
print(f"• Collecting {filename} results...")
r = campaign.get_results_as_dataframe(get_energy, verbose=True, parallel_parsing=True)
r["TotalEnergyWh"] = r["TotalEnergyJ"].to_numpy() / 3600
r.to_csv(cwd + f"/{filename}.csv")
r

### Total energy consumption [Wh]

In [ ]:
filename = "energy"
print(f"• Generating {filename} plot...")
# Load and filter data
df = pd.read_csv(cwd + "/energy.csv")
for o, n in zip(["SF7Single", "FullRand", "Classic"], ["SCSF7", "Random", "LoRaWAN"]):
    df.loc[df["scenario"] == o, "scenario"] = n
## Plot data
fig, ax = plt.subplots(figsize=plot_size(3))
sns.lineplot(
    data=df,
    y="TotalEnergyWh",
    x="gwNum",
    hue="scenario",
    style="scenario",
    err_style="bars",
    hue_order=["SCSF7", "Random", "LoRaWAN"],
    style_order=["SCSF7", "Random", "LoRaWAN"],
)
# Global settings
ax.set_xlabel("Number of gateways")
ax.set_ylabel("Energy consumption [Wh]")
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[::-1], labels[::-1], title=None)
ax.set_xticks(
    range(1, 11, 1),
)
ax.xaxis.minorticks_off()
# Export
plt.tight_layout()
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "pdf"))
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "png"))